![image info](https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/images/banner_1.png)

# Disponibilización de modelos

En este notebook aprenderá a guardar un modelo y a disponibilizarlo como una API con la librería Flask. Una API (interfaz de programación de aplicaciones) es un conjunto de definiciones y protocolos que permiten que servicios, en este caso modelos, retornen resultados y respuestas sin necesidad de saber cómo están implementados.

## Instrucciones Generales:

Este notebook esta compuesto por dos secciones. En la primera secciónn, usted beberá entrenar y guardar (exportar) un modelo de random forest para predecir si una URL es phishing (fraudulenta) o no. En la segunda parte, usará el modelo entrenado y lo disponibilizara usando la libreria *Flask*. En el siguente paper puede conocer más detalles de la base de datos que usaremos y del problema: *A. Correa Bahnsen, E. C. Bohorquez, S. Villegas, J. Vargas, and F. A. Gonzalez, “Classifying phishing urls using recurrent neural networks,” in Electronic Crime Research (eCrime), 2017 APWG Symposium on. IEEE, 2017, pp. 1–8*. https://albahnsen.files.wordpress.com/2018/05/classifying-phishing-urls-using-recurrent-neural-networks_cameraready.pdf
  
Para realizar la actividad, solo siga las indicaciones asociadas a cada celda del notebook. 


## Importar base de datos y librerías

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importación librerías
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import joblib

import os
os.chdir('..')

In [3]:
# Carga de datos de archivos .csv
data = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/phishing.csv')
data.head()

,url,phishing
0,http://www.subalipack.com/contact/images/sampl...,1
1,http://fasc.maximecapellot-gypsyjazz-ensemble....,1
2,http://theotheragency.com/confirmer/confirmer-...,1
3,http://aaalandscaping.com/components/com_smart...,1
4,http://paypal.com.confirm-key-21107316126168.s...,1


## Codificar variables categóricas
Relizar preprocesamiento de texto (URLs) para crear variables predictoras:

In [4]:
# Creación de columnas binarias que indican si la URL contiene la palabra clave (keywords)
keywords = ['https', 'login', '.php', '.html', '@', 'sign']
for keyword in keywords:
    data['keyword_' + keyword] = data.url.str.contains(keyword).astype(int)

# Definición de la variable largo de la URL
data['lenght'] = data.url.str.len() - 2

# Definición de la variable largo del dominio de la URL
domain = data.url.str.split('/', expand=True).iloc[:, 2]
data['lenght_domain'] = domain.str.len()

# Definición de la variable binaria que indica si es IP
data['isIP'] = (domain.str.replace('.', '') * 1).str.isnumeric().astype(int)

# Definicón de la variable cuenta de 'com' en la URL
data['count_com'] = data.url.str.count('com')

data.head()

,url,phishing,keyword_https,keyword_login,keyword_.php,keyword_.html,keyword_@,keyword_sign,lenght,lenght_domain,isIP,count_com
0,http://www.subalipack.com/contact/images/sampl...,1,0,0,0,0,0,0,47,18,0,1
1,http://fasc.maximecapellot-gypsyjazz-ensemble....,1,0,0,0,0,0,0,73,41,0,0
2,http://theotheragency.com/confirmer/confirmer-...,1,0,0,0,0,0,0,92,18,0,1
3,http://aaalandscaping.com/components/com_smart...,1,0,0,0,0,0,0,172,18,0,3
4,http://paypal.com.confirm-key-21107316126168.s...,1,0,0,0,0,0,0,90,50,0,1


In [5]:
# Separación de variables predictoras (X) y variable de interes (y)
X = data.drop(['url', 'phishing'], axis=1)
y = data.phishing

## Entrenar y guardar el modelo

In [6]:
# Definición de modelo de clasificación Random Forest
clf = RandomForestClassifier(n_jobs=-1, n_estimators=100, max_depth=3)
cross_val_score(clf, X, y, cv=10)

array([0.75075, 0.76125, 0.74475, 0.75425, 0.7545 , 0.76625, 0.75925,
       0.74925, 0.752  , 0.758  ])

In [7]:
# Entrenamiento del modelo de clasificación Random Forest
clf.fit(X, y)

RandomForestClassifier(max_depth=3, n_jobs=-1)

In [16]:
# Exportar modelo a archivo binario .pkl
joblib.dump(clf, 'model_deployment/phishing_clf.pkl', compress=3)

['model_deployment/phishing_clf.pkl']

In [17]:
# Importar modelo y predicción
from model_deployment.m09_model_deployment import predict_proba

# Predicción de probabilidad de que un link sea phishing
predict_proba('http://www.vipturismolondres.com/com.br/?atendimento=Cliente&/LgSgkszm64/B8aNzHa8Aj.php')

0.7626112730428848

## Disponibilizar modelo con Flask

Para esta sección del notebook instale las siguientes librerías *!pip install flask* y *!pip install flask_restplus*.

In [18]:
# Importación librerías
from flask import Flask
from flask_restx import Api, Resource, fields

In [49]:
# Definición aplicación Flask
from werkzeug.datastructures import FileStorage

app = Flask(__name__)

# Definición API Flask
api = Api(
    app, 
    version='1.0', 
    title='Phishing Prediction API',
    description='Phishing Prediction API')

ns = api.namespace('predict', 
     description='Phishing Classifier')

# Definición argumentos o parámetros de la API
parser = api.parser()
parser.add_argument(
    'file', 
    type=FileStorage, 
    required=True, 
    help='CSV file containing data to be analyzed', 
    location='files'
)



resource_fields = api.model('Resource', {
    'result': fields.String,
})

In [53]:
# Ejecución de la aplicación que disponibiliza el modelo de manera local en el puerto 5000
app.run(debug=True, use_reloader=False, host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.100.153:5000
Press CTRL+C to quit
192.168.100.153 - - [24/Apr/2024 20:36:43] "GET / HTTP/1.1" 200 -
192.168.100.153 - - [24/Apr/2024 20:36:43] "GET /swaggerui/droid-sans.css HTTP/1.1" 304 -
192.168.100.153 - - [24/Apr/2024 20:36:43] "GET /swaggerui/swagger-ui-bundle.js HTTP/1.1" 304 -
192.168.100.153 - - [24/Apr/2024 20:36:43] "GET /swaggerui/swagger-ui-standalone-preset.js HTTP/1.1" 304 -
192.168.100.153 - - [24/Apr/2024 20:36:43] "GET /swaggerui/swagger-ui.css HTTP/1.1" 304 -
192.168.100.153 - - [24/Apr/2024 20:36:43] "GET /swagger.json HTTP/1.1" 200 -
192.168.100.153 - - [24/Apr/2024 20:40:13] "GET / HTTP/1.1" 200 -
192.168.100.153 - - [24/Apr/2024 20:40:13] "GET /swaggerui/droid-sans.css HTTP/1.1" 304 -
192.168.100.153 - - [24/Apr/2024 20:40:13] "GET /swaggerui/swagger-ui.css HTTP/1.1" 304 -
192.168.100.153 - - [24/Apr/2024 20:40:13] "GET /swaggerui/swagger-ui-bundle.js HTTP/1.1" 

El modelo debe haber quedado disponibilizado en el puerto 5000. Para predecir la probabilidad de que una URL sea fraudulenta (phishing) copie en la barra de busqueda de su navegador la siguiente dirección (http://localhost:5000/predict/?URL=) y agregregue al final de esta la URL que desee precir. Por ejemplo, al copiar la URL http://localhost:5000/predict/?URL=http://consultoriojuridico.co/pp/www.paypal.com/, la API retornará la probabilidad de que la URL http://consultoriojuridico.co/pp/www.paypal.com/ sea phishing.

## CON ESTE ME FUNCIONO LA API DIRECTAMENTE EN MI PC LOCAL

In [108]:
# ESTA ES UNA API QUE PERMITE CARGAR UN ARCHIVO CSV Y PREDECIR
from flask import Flask, request
from flask_restx import Api, Resource, fields, reqparse
from werkzeug.datastructures import FileStorage
import pandas as pd

app = Flask(__name__)
api = Api(
    app, 
    version='1.0', 
    title='Phishing Prediction API',
    description='Phishing Prediction API'
)

ns = api.namespace('predict', description='Phishing Classifier')

# Definición de campos de respuesta
resource_fields = api.model('Resource', {
    'Prediccion_precio': fields.List(fields.Float),
})

# Función para procesar el archivo CSV y sumar las filas
def predict_proba(file):
    import pandas as pd
    from xgboost import XGBRegressor
    from sklearn.metrics import mean_squared_error
    # Transformacion de variables categoricas - One-Hot Encoding

    dataTraining = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/dataTrain_carListings.zip')
    dataTesting = pd.read_csv(file, index_col=0)

    # Datos X - y
    X = dataTraining.drop(columns=['Price'])
    y = dataTraining['Price']

    # Aplicar One-Hot Encoding a las variables categóricas
    XTrain_encoded = pd.get_dummies(X, dtype=int)


    # Definir el modelo XGBoost para un problema de regresión con paralelismo
    clf = XGBRegressor(n_estimators = 600, learning_rate = 0.1, max_depth = 15, reg_lambda=20, n_jobs=-1)


    # Entrenar el modelo
    clf.fit(XTrain_encoded, y)
    
    # Aplicar One-Hot Encoding a las variables categóricas en dataset Testing
    indice = XTrain_encoded.columns.get_loc("Make_Freightliner")
    XTest_encoded_final = pd.get_dummies(dataTesting, dtype=int)
    # Obtener la lista de columnas de XTrain_encoded y XTest_encoded_final
    columns_to_add = [col for col in XTrain_encoded.columns if col not in XTest_encoded_final.columns]

    # Agregar las columnas faltantes a XTest_encoded_final con valores 0
    for col in columns_to_add:
        XTest_encoded_final[col] = 0

    # Asegurarse de que las columnas estén en el mismo orden que en XTrain_encoded
    XTest_encoded_final = XTest_encoded_final[XTrain_encoded.columns]
    ## Predicciones XGboost - Test
    y_pred = clf.predict(XTest_encoded_final)
    y_pred = pd.DataFrame(y_pred)
    y_pred.columns = ['Price']
    lista = y_pred.values.tolist()
    lista_de_decimales = [numero for sublista in lista for numero in sublista]
    return lista_de_decimales

# Definición de argumentos para la API
parser = reqparse.RequestParser()
parser.add_argument(
    'file', 
    type=FileStorage, 
    required=True, 
    help='Archivo CSV con los datos a predecir', 
    location='files'
)


# Definición de la clase para disponibilización
@ns.route('/')
class PhishingApi(Resource):

    @api.doc()
    @api.expect(parser)
    @api.marshal_with(resource_fields)
    def post(self):
        args = parser.parse_args()
        file = args['file']

        # Llama a la función predicciones_modelo para procesar el archivo CSV
        suma = predict_proba(file)

        # Devuelve el resultado como respuesta JSON
        return {'Prediccion_precio': suma}, 200

# Inicia el servidor Flask
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False, host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.100.153:5000
Press CTRL+C to quit
192.168.100.153 - - [24/Apr/2024 23:37:27] "GET / HTTP/1.1" 200 -
192.168.100.153 - - [24/Apr/2024 23:37:27] "GET /swaggerui/droid-sans.css HTTP/1.1" 304 -
192.168.100.153 - - [24/Apr/2024 23:37:27] "GET /swaggerui/swagger-ui.css HTTP/1.1" 304 -
192.168.100.153 - - [24/Apr/2024 23:37:27] "GET /swaggerui/swagger-ui-bundle.js HTTP/1.1" 304 -
192.168.100.153 - - [24/Apr/2024 23:37:27] "GET /swaggerui/swagger-ui-standalone-preset.js HTTP/1.1" 304 -
192.168.100.153 - - [24/Apr/2024 23:37:27] "GET /swagger.json HTTP/1.1" 200 -
192.168.100.153 - - [24/Apr/2024 23:38:29] "POST /predict/ HTTP/1.1" 200 -


In [7]:
# ESTA API PERMITE CARGAR LOS CAMPOS POR SEPARADO Y PREDICIR DE A UN VALOR A LA VEZ
from flask import Flask
from flask_restx import Api, Resource, fields, reqparse
import pandas as pd

app = Flask(__name__)
api = Api(
    app, 
    version='1.0', 
    title='Car Price Prediction API',
    description='Car Price Prediction API'
)

ns = api.namespace('predict', description='Car Price Prediction')

# Definición de campos de respuesta
resource_fields = api.model('Resource', {
    'Prediction': fields.List(fields.Float),
})

# Función para realizar la predicción
def predict_price(year, mileage, state, make, model):
    from xgboost import XGBRegressor
    data = {
        'Year': [year],
        'Mileage': [mileage],
        'State': [state],
        'Make': [make],
        'Model': [model]
    }
    
    df = pd.DataFrame(data)
    df = pd.get_dummies(df, dtype=int)
    
    dataTraining = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/dataTrain_carListings.zip')
    
    # Datos X - y
    X = dataTraining.drop(columns=['Price'])
    y = dataTraining['Price']

    # Aplicar One-Hot Encoding a las variables categóricas
    XTrain_encoded = pd.get_dummies(X, dtype=int)


    # Definir el modelo XGBoost para un problema de regresión con paralelismo
    clf = XGBRegressor(n_estimators = 600, learning_rate = 0.1, max_depth = 15, reg_lambda=20, n_jobs=-1)


    # Entrenar el modelo
    clf.fit(XTrain_encoded, y)

    # Obtener la lista de columnas de XTrain_encoded y XTest_encoded_final
    columns_to_add = [col for col in XTrain_encoded.columns if col not in df.columns]

    # Agregar las columnas faltantes a XTest_encoded_final con valores 0
    for col in columns_to_add:
        df[col] = 0

    # Asegurarse de que las columnas estén en el mismo orden que en XTrain_encoded
    df = df[XTrain_encoded.columns]
    ## Predicciones XGboost - Test
    y_pred = clf.predict(df)
    y_pred = pd.DataFrame(y_pred)
    y_pred.columns = ['Price']
    predic_num = y_pred.values[0][0]
    return predic_num

# Definición de argumentos para la API
parser = reqparse.RequestParser()
parser.add_argument('Year', type=int, required=True, help='Vehicle Year')
parser.add_argument('Mileage', type=int, required=True, help='Vehicle Mileage')
parser.add_argument('State', type=str, required=True, help='State of the vehicle')
parser.add_argument('Make', type=str, required=True, help='Make of the vehicle')
parser.add_argument('Model', type=str, required=True, help='Model of the vehicle')

# Definición de la clase para la API
@ns.route('/')
class CarPriceApi(Resource):

    @api.doc(parser=parser)
    @api.marshal_with(resource_fields)
    def post(self):
        args = parser.parse_args()

        # Llama a la función predict_price para realizar la predicción
        prediction = predict_price(args['Year'], args['Mileage'], args['State'], args['Make'], args['Model'])

        # Devuelve el resultado como respuesta JSON
        return {'Prediction': [prediction]}, 200

# Inicia el servidor Flask
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False, host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.100.153:5000
Press CTRL+C to quit
192.168.100.153 - - [25/Apr/2024 03:10:46] "GET / HTTP/1.1" 200 -
192.168.100.153 - - [25/Apr/2024 03:10:46] "GET /swaggerui/droid-sans.css HTTP/1.1" 304 -
192.168.100.153 - - [25/Apr/2024 03:10:46] "GET /swaggerui/swagger-ui.css HTTP/1.1" 304 -
192.168.100.153 - - [25/Apr/2024 03:10:46] "GET /swaggerui/swagger-ui-bundle.js HTTP/1.1" 304 -
192.168.100.153 - - [25/Apr/2024 03:10:46] "GET /swaggerui/swagger-ui-standalone-preset.js HTTP/1.1" 304 -
192.168.100.153 - - [25/Apr/2024 03:10:47] "GET /swagger.json HTTP/1.1" 200 -
C:\Users\U1037319\AppData\Local\Temp\ipykernel_6308\3608330786.py:57: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = fra

In [10]:
# ESTA API PERMITE CARGAR LOS CAMPOS POR SEPARADO Y PREDICIR DE A UN VALOR A LA VEZ
from flask import Flask
from flask_restx import Api, Resource, fields, reqparse
import pandas as pd

app = Flask(__name__)
api = Api(
    app, 
    version='1.0', 
    title='Car Price Prediction API',
    description='Car Price Prediction API'
)

ns = api.namespace('predict', description='Car Price Prediction')

# Definición de campos de respuesta
resource_fields = api.model('Resource', {
    'Prediction': fields.List(fields.Float),
})

# Función para realizar la predicción
def predict_price(year, mileage, state, make, model):
    XTrain_encoded =  ['Year','Mileage','State_ AK','State_ AL','State_ AR','State_ AZ','State_ CA','State_ CO','State_ CT','State_ DC','State_ DE','State_ FL','State_ GA','State_ HI','State_ IA','State_ ID','State_ IL','State_ IN','State_ KS','State_ KY','State_ LA','State_ MA','State_ MD','State_ ME','State_ MI','State_ MN','State_ MO','State_ MS','State_ MT','State_ NC','State_ ND','State_ NE','State_ NH','State_ NJ','State_ NM','State_ NV','State_ NY','State_ OH','State_ OK','State_ OR','State_ PA','State_ RI','State_ SC','State_ SD','State_ TN','State_ TX','State_ UT','State_ VA','State_ VT','State_ WA','State_ WI','State_ WV','State_ WY','Make_Acura','Make_Audi','Make_BMW','Make_Bentley','Make_Buick','Make_Cadillac','Make_Chevrolet','Make_Chrysler','Make_Dodge','Make_FIAT','Make_Ford','Make_Freightliner','Make_GMC','Make_Honda','Make_Hyundai','Make_INFINITI','Make_Jaguar','Make_Jeep','Make_Kia','Make_Land','Make_Lexus','Make_Lincoln','Make_MINI','Make_Mazda','Make_Mercedes-Benz','Make_Mercury','Make_Mitsubishi','Make_Nissan','Make_Pontiac','Make_Porsche','Make_Ram','Make_Scion','Make_Subaru','Make_Suzuki','Make_Tesla','Make_Toyota','Make_Volkswagen','Make_Volvo','Model_1','Model_15002WD','Model_15004WD','Model_1500Laramie','Model_1500Tradesman','Model_200LX','Model_200Limited','Model_200S','Model_200Touring','Model_25002WD','Model_25004WD','Model_3','Model_300300C','Model_300300S','Model_3004dr','Model_300Base','Model_300Limited','Model_300Touring','Model_35004WD','Model_350Z2dr','Model_4Runner2WD','Model_4Runner4WD','Model_4Runner4dr','Model_4RunnerLimited','Model_4RunnerRWD','Model_4RunnerSR5','Model_4RunnerTrail','Model_5','Model_500Pop','Model_6','Model_7','Model_911','Model_9112dr','Model_A34dr','Model_A44dr','Model_A64dr','Model_A8','Model_AcadiaAWD','Model_AcadiaFWD','Model_Accent4dr','Model_Accord','Model_AccordEX','Model_AccordEX-L','Model_AccordLX','Model_AccordLX-S','Model_AccordSE','Model_Altima4dr','Model_Armada2WD','Model_Armada4WD','Model_Avalanche2WD','Model_Avalanche4WD','Model_Avalon4dr','Model_AvalonLimited','Model_AvalonTouring','Model_AvalonXLE','Model_Azera4dr','Model_Boxster2dr','Model_C-Class4dr','Model_C-ClassC','Model_C-ClassC300','Model_C-ClassC350','Model_C702dr','Model_CC4dr','Model_CR-V2WD','Model_CR-V4WD','Model_CR-VEX','Model_CR-VEX-L','Model_CR-VLX','Model_CR-VSE','Model_CR-ZEX','Model_CT','Model_CTCT','Model_CTS','Model_CTS-V','Model_CTS4dr','Model_CX-7FWD','Model_CX-9AWD','Model_CX-9FWD','Model_CX-9Grand','Model_CX-9Touring','Model_Caliber4dr','Model_Camaro2dr','Model_CamaroConvertible','Model_CamaroCoupe','Model_Camry','Model_Camry4dr','Model_CamryBase','Model_CamryL','Model_CamryLE','Model_CamrySE','Model_CamryXLE','Model_Canyon2WD','Model_Canyon4WD','Model_CanyonCrew','Model_CanyonExtended','Model_CayenneAWD','Model_Cayman2dr','Model_Challenger2dr','Model_ChallengerR/T','Model_Charger4dr','Model_ChargerSE','Model_ChargerSXT','Model_CherokeeLimited','Model_CherokeeSport','Model_Civic','Model_CivicEX','Model_CivicEX-L','Model_CivicLX','Model_CivicSi','Model_Cobalt2dr','Model_Cobalt4dr','Model_Colorado2WD','Model_Colorado4WD','Model_ColoradoCrew','Model_ColoradoExtended','Model_Compass4WD','Model_CompassLatitude','Model_CompassLimited','Model_CompassSport','Model_Continental','Model_Cooper','Model_Corolla4dr','Model_CorollaL','Model_CorollaLE','Model_CorollaS','Model_Corvette2dr','Model_CorvetteConvertible','Model_CorvetteCoupe','Model_CruzeLT','Model_CruzeSedan','Model_DTS4dr','Model_Dakota2WD','Model_Dakota4WD','Model_Durango2WD','Model_Durango4dr','Model_DurangoAWD','Model_DurangoSXT','Model_E-ClassE','Model_E-ClassE320','Model_E-ClassE350','Model_ES','Model_ESES','Model_Eclipse3dr','Model_Econoline','Model_EdgeLimited','Model_EdgeSE','Model_EdgeSEL','Model_EdgeSport','Model_Elantra','Model_Elantra4dr','Model_ElantraLimited','Model_Element2WD','Model_Element4WD','Model_EnclaveConvenience','Model_EnclaveLeather','Model_EnclavePremium','Model_Eos2dr','Model_EquinoxAWD','Model_EquinoxFWD','Model_Escalade','Model_Escalade2WD','Model_Escalade4dr','Model_EscaladeAWD','Model_Escape4WD','Model_Escape4dr','Model_EscapeFWD','Model_EscapeLImited','Model_EscapeLimited','Model_EscapeS','Model_EscapeSE','Model_EscapeXLT','Model_Excursion137"','Model_Expedition','Model_Expedition2WD','Model_Expedition4WD','Model_ExpeditionLimited','Model_ExpeditionXLT','Model_Explorer','Model_Explorer4WD','Model_Explorer4dr','Model_ExplorerBase','Model_ExplorerEddie','Model_ExplorerFWD','Model_ExplorerLimited','Model_ExplorerXLT','Model_Express','Model_F-1502WD','Model_F-1504WD','Model_F-150FX2','Model_F-150FX4','Model_F-150King','Model_F-150Lariat','Model_F-150Limited','Model_F-150Platinum','Model_F-150STX','Model_F-150SuperCrew','Model_F-150XL','Model_F-150XLT','Model_F-250King','Model_F-250Lariat','Model_F-250XL','Model_F-250XLT','Model_F-350King','Model_F-350Lariat','Model_F-350XL','Model_F-350XLT','Model_FJ','Model_FX35AWD','Model_FiestaS','Model_FiestaSE','Model_FitSport','Model_FlexLimited','Model_FlexSE','Model_FlexSEL','Model_Focus4dr','Model_Focus5dr','Model_FocusS','Model_FocusSE','Model_FocusSEL','Model_FocusST','Model_FocusTitanium','Model_Forester2.5X','Model_Forester4dr','Model_Forte','Model_ForteEX','Model_ForteLX','Model_ForteSX','Model_Frontier','Model_Frontier2WD','Model_Frontier4WD','Model_Fusion4dr','Model_FusionHybrid','Model_FusionS','Model_FusionSE','Model_FusionSEL','Model_G35','Model_G37','Model_G64dr','Model_GLI4dr','Model_GS','Model_GSGS','Model_GTI2dr','Model_GTI4dr','Model_GX','Model_GXGX','Model_Galant4dr','Model_Genesis','Model_Golf','Model_Grand','Model_Highlander','Model_Highlander4WD','Model_Highlander4dr','Model_HighlanderBase','Model_HighlanderFWD','Model_HighlanderLimited','Model_HighlanderSE','Model_IS','Model_ISIS','Model_Impala4dr','Model_ImpalaLS','Model_ImpalaLT','Model_Impreza','Model_Impreza2.0i','Model_ImprezaSport','Model_Jetta','Model_JourneyAWD','Model_JourneyFWD','Model_JourneySXT','Model_LS','Model_LSLS','Model_LX','Model_LXLX','Model_LaCrosse4dr','Model_LaCrosseAWD','Model_LaCrosseFWD','Model_Lancer4dr','Model_Land','Model_Legacy','Model_Legacy2.5i','Model_Legacy3.6R','Model_Liberty4WD','Model_LibertyLimited','Model_LibertySport','Model_Lucerne4dr','Model_M-ClassML350','Model_MDX4WD','Model_MDXAWD','Model_MKXAWD','Model_MKXFWD','Model_MKZ4dr','Model_MX5','Model_Malibu','Model_Malibu1LT','Model_Malibu4dr','Model_MalibuLS','Model_MalibuLT','Model_Matrix5dr','Model_Maxima4dr','Model_Mazda34dr','Model_Mazda35dr','Model_Mazda64dr','Model_Milan4dr','Model_Model','Model_Monte','Model_Murano2WD','Model_MuranoAWD','Model_MuranoS','Model_Mustang2dr','Model_MustangBase','Model_MustangDeluxe','Model_MustangGT','Model_MustangPremium','Model_MustangShelby','Model_Navigator','Model_Navigator2WD','Model_Navigator4WD','Model_Navigator4dr','Model_New','Model_OdysseyEX','Model_OdysseyEX-L','Model_OdysseyLX','Model_OdysseyTouring','Model_Optima4dr','Model_OptimaEX','Model_OptimaLX','Model_OptimaSX','Model_Outback2.5i','Model_Outback3.6R','Model_Outlander','Model_Outlander2WD','Model_Outlander4WD','Model_PT','Model_PacificaLimited','Model_PacificaTouring','Model_Passat','Model_Passat4dr','Model_Pathfinder2WD','Model_Pathfinder4WD','Model_PathfinderS','Model_PathfinderSE','Model_Patriot4WD','Model_PatriotLatitude','Model_PatriotLimited','Model_PatriotSport','Model_Pilot2WD','Model_Pilot4WD','Model_PilotEX','Model_PilotEX-L','Model_PilotLX','Model_PilotSE','Model_PilotTouring','Model_Prius','Model_Prius5dr','Model_PriusBase','Model_PriusFive','Model_PriusFour','Model_PriusOne','Model_PriusThree','Model_PriusTwo','Model_Q5quattro','Model_Q7quattro','Model_QX562WD','Model_QX564WD','Model_Quest4dr','Model_RAV4','Model_RAV44WD','Model_RAV44dr','Model_RAV4Base','Model_RAV4FWD','Model_RAV4LE','Model_RAV4Limited','Model_RAV4Sport','Model_RAV4XLE','Model_RDXAWD','Model_RDXFWD','Model_RX','Model_RX-84dr','Model_RXRX','Model_Ram','Model_Ranger2WD','Model_Ranger4WD','Model_RangerSuperCab','Model_Regal4dr','Model_RegalGS','Model_RegalPremium','Model_RegalTurbo','Model_RidgelineRTL','Model_RidgelineSport','Model_RioLX','Model_RogueFWD','Model_Rover','Model_S2000Manual','Model_S44dr','Model_S60T5','Model_S804dr','Model_SC','Model_SL-ClassSL500','Model_SLK-ClassSLK350','Model_SRXLuxury','Model_STS4dr','Model_Santa','Model_Savana','Model_Sedona4dr','Model_SedonaEX','Model_SedonaLX','Model_Sentra4dr','Model_Sequoia4WD','Model_Sequoia4dr','Model_SequoiaLimited','Model_SequoiaPlatinum','Model_SequoiaSR5','Model_Sienna5dr','Model_SiennaLE','Model_SiennaLimited','Model_SiennaSE','Model_SiennaXLE','Model_Sierra','Model_Silverado','Model_Sonata4dr','Model_SonataLimited','Model_SonataSE','Model_SonicHatch','Model_SonicSedan','Model_Sorento2WD','Model_SorentoEX','Model_SorentoLX','Model_SorentoSX','Model_Soul+','Model_SoulBase','Model_Sportage2WD','Model_SportageAWD','Model_SportageEX','Model_SportageLX','Model_SportageSX','Model_Sprinter','Model_Suburban2WD','Model_Suburban4WD','Model_Suburban4dr','Model_Super','Model_TL4dr','Model_TLAutomatic','Model_TSXAutomatic','Model_TT2dr','Model_Tacoma2WD','Model_Tacoma4WD','Model_TacomaBase','Model_TacomaPreRunner','Model_Tahoe2WD','Model_Tahoe4WD','Model_Tahoe4dr','Model_TahoeLS','Model_TahoeLT','Model_Taurus4dr','Model_TaurusLimited','Model_TaurusSE','Model_TaurusSEL','Model_TaurusSHO','Model_TerrainAWD','Model_TerrainFWD','Model_Tiguan2WD','Model_TiguanS','Model_TiguanSE','Model_TiguanSEL','Model_Titan','Model_Titan2WD','Model_Titan4WD','Model_Touareg4dr','Model_Town','Model_Transit','Model_TraverseAWD','Model_TraverseFWD','Model_TucsonAWD','Model_TucsonFWD','Model_TucsonLimited','Model_Tundra','Model_Tundra2WD','Model_Tundra4WD','Model_TundraBase','Model_TundraLimited','Model_TundraSR5','Model_VeracruzAWD','Model_VeracruzFWD','Model_Versa4dr','Model_Versa5dr','Model_Vibe4dr','Model_WRXBase','Model_WRXLimited','Model_WRXPremium','Model_WRXSTI','Model_Wrangler','Model_Wrangler2dr','Model_Wrangler4WD','Model_WranglerRubicon','Model_WranglerSahara','Model_WranglerSport','Model_WranglerX','Model_X1xDrive28i','Model_X3AWD','Model_X3xDrive28i','Model_X5AWD','Model_X5xDrive35i','Model_XC60AWD','Model_XC60FWD','Model_XC60T6','Model_XC704dr','Model_XC90AWD','Model_XC90FWD','Model_XC90T6','Model_XF4dr','Model_XJ4dr','Model_XK2dr','Model_Xterra2WD','Model_Xterra4WD','Model_Xterra4dr','Model_Yaris','Model_Yaris4dr','Model_YarisBase','Model_YarisLE','Model_Yukon','Model_Yukon2WD','Model_Yukon4WD','Model_Yukon4dr','Model_tC2dr','Model_xB5dr','Model_xD5dr']
    data = {
        'Year': [year],
        'Mileage': [mileage],
        'State': [state],
        'Make': [make],
        'Model': [model]
    }
    
    df = pd.DataFrame(data)
    df = pd.get_dummies(df, dtype=int)
    
    # Define la ruta absoluta al archivo del modelo
    model_path = os.path.abspath('cars_rg.pkl')

    # Carga el modelo
    clf = joblib.load(model_path)

    # Obtener la lista de columnas de XTrain_encoded y XTest_encoded_final
    columns_to_add = [col for col in XTrain_encoded if col not in df.columns]

    # Agregar las columnas faltantes a XTest_encoded_final con valores 0
    for col in columns_to_add:
        df[col] = 0

    # Asegurarse de que las columnas estén en el mismo orden que en XTrain_encoded
    df = df[XTrain_encoded]
    ## Predicciones XGboost - Test
    y_pred = clf.predict(df)
    y_pred = pd.DataFrame(y_pred)
    y_pred.columns = ['Price']
    predic_num = y_pred.values[0][0]
    return predic_num

# Definición de argumentos para la API
parser = reqparse.RequestParser()
parser.add_argument('Year', type=int, required=True, help='Vehicle Year')
parser.add_argument('Mileage', type=int, required=True, help='Vehicle Mileage')
parser.add_argument('State', type=str, required=True, help='State of the vehicle')
parser.add_argument('Make', type=str, required=True, help='Make of the vehicle')
parser.add_argument('Model', type=str, required=True, help='Model of the vehicle')

# Definición de la clase para la API
@ns.route('/')
class CarPriceApi(Resource):

    @api.doc(parser=parser)
    @api.marshal_with(resource_fields)
    def post(self):
        args = parser.parse_args()

        # Llama a la función predict_price para realizar la predicción
        prediction = predict_price(args['Year'], args['Mileage'], args['State'], args['Make'], args['Model'])

        # Devuelve el resultado como respuesta JSON
        return {'Prediction': [prediction]}, 200

# Inicia el servidor Flask
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False, host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.100.153:5000
Press CTRL+C to quit
192.168.100.153 - - [25/Apr/2024 03:31:02] "GET / HTTP/1.1" 200 -
192.168.100.153 - - [25/Apr/2024 03:31:02] "GET /swaggerui/droid-sans.css HTTP/1.1" 304 -
192.168.100.153 - - [25/Apr/2024 03:31:02] "GET /swaggerui/swagger-ui.css HTTP/1.1" 304 -
192.168.100.153 - - [25/Apr/2024 03:31:02] "GET /swaggerui/swagger-ui-bundle.js HTTP/1.1" 304 -
192.168.100.153 - - [25/Apr/2024 03:31:02] "GET /swaggerui/swagger-ui-standalone-preset.js HTTP/1.1" 304 -
192.168.100.153 - - [25/Apr/2024 03:31:02] "GET /swagger.json HTTP/1.1" 200 -
C:\Users\U1037319\AppData\Local\Temp\ipykernel_6308\1646017620.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = fra